##Importación correcta de SMA

In [1]:
from core.db import get_engine
from core.libs import pd, text, np

engine = get_engine()

💻 Conectado a la base de datos helloworldtree


In [16]:
df_imc_null = pd.read_sql("""
    SELECT contract_code
    FROM masterdatabase.inventory_metrics_current
    WHERE survival IS NULL;
""", engine)

print("NULLs actuales en IMC:", len(df_imc_null))


NULLs actuales en IMC: 0


In [5]:
path = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\DatabaseExports\survival_nulls_classified.xlsx"

df_nulls = pd.read_excel(path, sheet_name="Sheet1")

df_nulls = df_nulls[[
    "contract_code",
    "ultimate_survival",
    "new_inventory_type"
]].copy()

print("NULLs en el Excel:", len(df_nulls))


NULLs en el Excel: 244


In [10]:
a = df_nulls[df_nulls["ultimate_survival"].isna()]
print(len(a))
print(a.head())

b = a.groupby("new_inventory_type").size()
print(b.head())


138
   contract_code  ultimate_survival    new_inventory_type
1         MX0099                NaN  Not planted contract
8         MX0123                NaN  Not planted contract
9         CR0031                NaN        out of program
10        CR0032                NaN        out of program
11        CR0023                NaN        out of program
new_inventory_type
Not planted contract     34
out of program          104
dtype: int64


In [13]:
df_merge = df_imc_null.merge(df_nulls, on="contract_code", how="left")

no_match = df_merge[df_merge["ultimate_survival"].isna()]
match = df_merge[df_merge["ultimate_survival"].notna()]

print(len(df_merge))
print(df_merge.head())
print("No match:", len(no_match))
print(no_match.head())
print("Match:", len(match))
print(match.head())


171
  contract_code  ultimate_survival new_inventory_type
0        CR0001                NaN     out of program
1        CR0002                NaN     out of program
2        CR0003                NaN     out of program
3        CR0004                NaN     out of program
4        CR0005                NaN     out of program
No match: 138
  contract_code  ultimate_survival new_inventory_type
0        CR0001                NaN     out of program
1        CR0002                NaN     out of program
2        CR0003                NaN     out of program
3        CR0004                NaN     out of program
4        CR0005                NaN     out of program
Match: 33
   contract_code  ultimate_survival        new_inventory_type
30        CR0036               0.83            out of program
32        CR0040               0.93  maindatabase last record
35        CR0045               0.83  maindatabase last record
38        CR0056               0.83            out of program
39        CR00

In [15]:
df_update = df_imc_null.merge(df_nulls, on="contract_code", how="left")

print("Filas preparadas para update:", len(df_update))


#===============================================================================
# Realizar los updates en la base de datos
#===============================================================================

#Va el paso de la muerte:

from sqlalchemy import text

query = text("""
    UPDATE masterdatabase.inventory_metrics_current
    SET
        survival = :survival,
        type_of_metric = :metric_type
    WHERE contract_code = :contract_code
""")

with engine.begin() as conn:
    for idx, row in df_update.iterrows():
        conn.execute(
            query,
            {
                "survival": row["ultimate_survival"],
                "metric_type": row["new_inventory_type"],
                "contract_code": row["contract_code"]
            }
        )



Filas preparadas para update: 171


##Comparación de SMA e IMC

In [18]:
#Importaciones

q_imc = """
    SELECT
        contract_code,
        survival AS imc_survival,
        total_trees AS imc_total_trees,
        inventory_year
    FROM masterdatabase.inventory_metrics_current
"""

df_imc = pd.read_sql(q_imc, engine)
print("IMC:", len(df_imc))


q_sma = """
    SELECT
        contract_code,
        survival_value AS sma_survival,
        trees_value AS sma_trees
    FROM masterdatabase.survival_maindatabase_archive
    WHERE source_label = 'LUKA_2024'
"""

df_sma = pd.read_sql(q_sma, engine)
print("SMA:", len(df_sma))


q_cti = """
    SELECT
        contract_code,
        status,
        planting_year,
        etp_year,
        region
    FROM masterdatabase.contract_tree_information
"""

df_cti = pd.read_sql(q_cti, engine)
print("CTI:", len(df_cti))


IMC: 462
SMA: 423
CTI: 462


In [21]:
df = (
    df_imc
    .merge(df_sma, on="contract_code", how="left")
    .merge(df_cti, on="contract_code", how="left")
)

df["diff"] = df["imc_survival"] - df["sma_survival"]


In [24]:
def classify_row(row):
    status = row["status"]
    inv_year = row["inventory_year"]
    diff = row["diff"]

    # Case 1: Out of Program
    if status == "Out of Program":
        return "Case 1: Out of program"

    # Case 2: New inventory 2025
    if inv_year == 2025:
        return "Case 2: New inventory 2025"

    # Case 3: IMC never imported previously (inventory_year is NaN)
    if pd.isna(inv_year):
        return "Case 3: Not imported previously"

    # Case 4a: Rounding error hundredths
    if abs(diff) < 0.001:
        return "Case 4a: Rounding error hundredths"

    # Case 4b: Rounding error tenths
    if abs(diff) < 0.01:
        return "Case 4b: Rounding error tenths"

    # Case 4c: Rounding error 5%
    if abs(diff) < 0.05:
        return "Case 4c: Rounding error 5%"

    # Default → significant discrepancy
    return "Case X: Significant discrepancy"


In [28]:
df["classification"] = df.apply(classify_row, axis=1)

df.columns

df["classification"].value_counts(dropna=False)


classification
Case 1: Out of program                139
Case 3: Not imported previously       137
Case 2: New inventory 2025             88
Case X: Significant discrepancy        32
Case 4b: Rounding error tenths         26
Case 4a: Rounding error hundredths     26
Case 4c: Rounding error 5%             14
Name: count, dtype: int64